In [ ]:
import os
import utils
import connectome_create
# viz_method = one of ['itkwidgets', 'vtk']
viz_method = 'vtk'

# import some of our favorite packages
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
import re
import warnings

%matplotlib inline
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.cluster import AgglomerativeClustering
import nglui.statebuilder as ngstbld

# this is the EM specific package for querying the EM data
from caveclient import CAVEclient


from meshparty import trimesh_io, trimesh_vtk
from meshparty import skeletonize, skeleton_io, skeleton
import cloudvolume

%load_ext autoreload
%autoreload 2

In [ ]:
# client = CAVEclient()

# # if not os.path.isfile(os.path.expanduser("~/.cloudvolume/secrets/cave-secret.json")):
# client.auth.get_new_token(open=True)

In [ ]:
# # if you have not yet setup this computer, uncomment this below line
# # paste the token from the website in, and run the line

# client.auth.save_token(token="4c0043f50c4a6ec89c05957d22ab8453", overwrite=True)

# # then comment or delete the line as you don't need to run it on this computer  again

In [ ]:
# from pathlib import Path
# import json

# with open(Path.home() / '.cloudvolume/secrets/'/'cave-secret.json') as f:
#         tokens = json.load(f)
        
# seg_source = 'graphene://https://cave.fanc-fly.com/segmentation/table/mar2021_prod'
# cv = cloudvolume.CloudVolume(cloudpath=seg_source, use_https=True, secrets=tokens)

In [ ]:
datastack_name = 'fanc_production_mar2021'

client = CAVEclient(datastack_name)

# Load Premotor connectivity matrix

In [ ]:
pre_to_mn_df = connectome_create.load_pre_to_mn_df(ext='ordered')


In [ ]:
pre_to_mn_df

# Test suite

In [ ]:
All = slice(None)

# # How many motor neurons are presynaptic to motor neurons
# # segID	motor	has_soma	sensory	 neck	local	
motor = pre_to_mn_df.loc[(All,True,True,All,False,False),:]
# descending.index.get_level_values('segID').to_list()
motor.shape

# How many sensory neurons # 94
# segID	motor	has_soma	sensory	 neck	local	
sensory = pre_to_mn_df.loc[(All,False,False,True,False,False),:]
# descending.index.get_level_values('segID').to_list()
sensory.shape

# How many descending neurons # 217 214?
# segID	motor	has_soma	sensory	 neck	local	
descending = pre_to_mn_df.loc[(All,False,False,False,True),:]
# descending.index.get_level_values('segID').to_list()
descending.shape

# How many motor neurons are presynaptic to motor neurons
# segID	motor	has_soma	sensory	 neck	local	
ascending = pre_to_mn_df.loc[(All,False,True,False,True,False),:]
# descending.index.get_level_values('segID').to_list()
ascending.head()
# some ascending that should be ascending
# 648518346507084872
# 648518346480786912
# 648518346490374748
sid = 648518346504729203
ascending.index.get_level_values('segID')==sid
# All are here

# Fragments?
# segID	motor	has_soma	sensory	 neck	local	
sensmns = pre_to_mn_df.loc[(All,False,True,True,False,False),:]
sensmns.shape

# Fragments?
# segID	motor	has_soma	sensory	 neck	local	
fragments = pre_to_mn_df.loc[(All,False,False,False,False,False),:]
fragments.shape

# Local and intersegmental? # 880
# segID	motor	has_soma	sensory	 neck	local	
vncns = pre_to_mn_df.loc[(All,False,True,False,False,False),:]
# descending.index.get_level_values('segID').to_list()
vncns.shape


# Find intersegmental neurons
Go through the vnc premotor neurons and look for a synapse outside the left hand box.
Once you find one, move to the next premotor neuron

In [ ]:
# Soma box
soma_table = client.materialize.query_table('somas_dec2022',timestamp=connectome_create.get_timestamp())
soma_table.index = soma_table.pt_root_id

In [ ]:
sx = [-3000, 3000]
sy = [-3000, 3000]
sz = [-300, 300]

# T1 bounding box, click on neuroglancer instance
All = slice(None)
x = [3000 , 38000]
y = [90483, 123190] # extended the boundaries a bit 5/10/22
z = [980, 3858]

t1 = lambda b: (b[0]>=x[0]) & (b[0]<=x[1]) & (b[1]>=y[0]) & (b[1]<=y[1]) & (b[2]>=z[0]) & (b[2]<=z[1])

def get_non_T1_inputs(iln_syn_df, s):
    in_T1_box = iln_syn_df.loc[:,['post_pt_position']].applymap(t1)
    pts_out_of_T1 = iln_syn_df.loc[~in_T1_box.post_pt_position,:]
    pts_in_T1 = iln_syn_df.loc[in_T1_box.post_pt_position,:]
    nearsoma = lambda b: (b[0]>=s[0]+sx[0]) & (b[0]<=s[0]+sx[1]) & (b[1]>=s[1]+sy[0]) & (b[1]<=s[1]+sy[1]) & (b[2]>=s[2]+sz[0]) & (b[2]<=s[2]+sz[1])
    in_soma_box = pts_out_of_T1.loc[:,['post_pt_position']].applymap(nearsoma)
    pts_out_of_T1 = pts_out_of_T1.loc[~in_soma_box.post_pt_position,:]
    return pts_out_of_T1

In [ ]:
from datetime import date
vncnsindex = vncns.index.to_frame()
vncnsindex.index = vncnsindex.index.get_level_values('segID')
vncnsindex['non_T1_count'] = 0

cnt = 0
for idx,row in vncnsindex.iterrows():
    cnt = cnt+1
    sid = row.segID
    loctuple = (row.segID,row.motor,row.has_soma,row.sensory,row.neck,row.local)
    synapsecnt = vncns.loc[loctuple,:].sum()
    iln_syn_df = client.materialize.synapse_query(post_ids = sid,timestamp=connectome_create.get_timestamp())
    # iln_syn_df = client.materialize.query_table('synapses_nov2022',
    #                                             filter_equal_dict = {'post_pt_root_id': sid},
    #                                             select_columns=['id','post_pt_position'],
    #                                             timestamp=connectome_create.get_timestamp()
    #                                                )
    
    try:
        s=soma_table.loc[sid,:].pt_position
    except KeyError:
        if 'fraglist' not in locals():
            today = date.today()
            d1 = today.strftime("%Y%m%d")
            fn = './dfs_saved/' + 'frag_df' + '_' + d1 + '.pkl'
            fraglist= pd.read_pickle(fn)
        if sid in fraglist.pt_root_id.to_list():
                print('{} is a known fragment of a local neuron'.format(sid))
                vncnsindex.loc[idx,'local'] = True
    if (type(s) is not np.ndarray) and (s.shape[0] != 3):
        print('{} has {} somas idx'.format(sid,s.shape[0]))
        continue
    non_T1_inputs = get_non_T1_inputs(iln_syn_df, s)
    if non_T1_inputs.empty:
        #print('{} is local'.format(sid))
        vncnsindex.loc[idx,'local'] = True
    else: 
        #print('{} has {} inputs outside of T1'.format(sid,non_T1_inputs.shape[0]))
        vncnsindex.loc[idx,'non_T1_count'] = non_T1_inputs.shape[0]
    if cnt % 100 == 0:
        print('{} of {} rows complete, idx = {}'.format(cnt,vncnsindex.shape[0],idx))

vncns.index = pd.MultiIndex.from_frame(vncnsindex)

# Reconcatenate dataframes

In [ ]:
sensmns.head(15)

In [ ]:
mnindex= motor.index.to_frame()
mnindex.index = mnindex.index.get_level_values('segID')
mnindex['non_T1_count'] = 0
motor.index = pd.MultiIndex.from_frame(mnindex)

descindex= descending.index.to_frame()
descindex.index = descindex.index.get_level_values('segID')
descindex['non_T1_count'] = 0
descending.index = pd.MultiIndex.from_frame(descindex)

sensoryindex= sensory.index.to_frame()
sensoryindex.index = sensoryindex.index.get_level_values('segID')
sensoryindex['non_T1_count'] = 0
sensory.index = pd.MultiIndex.from_frame(sensoryindex)

ascindex= ascending.index.to_frame()
ascindex.index = ascindex.index.get_level_values('segID')
ascindex['non_T1_count'] = 0
ascending.index = pd.MultiIndex.from_frame(ascindex)

sensmnsindex= sensmns.index.to_frame()
sensmnsindex.index = sensmnsindex.index.get_level_values('segID')
sensmnsindex['non_T1_count'] = 0
sensmns.index = pd.MultiIndex.from_frame(sensmnsindex)

frgindex= fragments.index.to_frame()
frgindex.index = frgindex.index.get_level_values('segID')
frgindex['non_T1_count'] = 0
fragments.index = pd.MultiIndex.from_frame(frgindex)

ordered_pMN_to_MN_df = pd.concat([
    motor,
    descending,
    sensory,
    vncns,
    ascending,
    sensmns,
    fragments,
])

In [ ]:
# total number of neurons with somas. Does this make sense?
# segID	motor	has_soma	sensory	 neck	local	
print(pre_to_mn_df.loc[(All,All,True,All,All),:].shape)
ordered_pMN_to_MN_df.loc[(All,All,True,All,All),:].shape

In [ ]:
ordered_pMN_to_MN_df.head()

In [ ]:
connectome_create.save_pre_to_mn_df(ordered_pMN_to_MN_df,ext='vncns_inter_vs_local')

In [ ]:
# If a segID shows up later, figure out what's going on here

# Create local vs intersegmental dfs.

# Local

In [ ]:
local = vncns.loc[(All,False,True,False,False,True,0),:]
print(local.shape)

intrseg = vncns.loc[(All,False,True,False,False,False,All),:]
print(intrseg.shape)
utils.total_sort_save_df_csv(intrseg,'intersegmental_ns')

### Is a segID intersegmental?

In [ ]:
# def build_synapses(sid,syn_df):

#     # variables to render in neuroglancer link
#     render_synapses = syn_df
#     render_neurons = [sid]

#     img_source = client.info.image_source()
#     img_layer = ngstbld.ImageLayerConfig(name='fanc_v4',
#                                  source=img_source,
#                                  )
#     seg_source = client.info.segmentation_source()

#     seg_layer = ngstbld.SegmentationLayerConfig(name = 'seg',
#                                         source = seg_source,
#                                         fixed_ids = render_neurons)
#     points = ngstbld.PointMapper(point_column='post_pt_position') ######################## change this to toggle rendering of pre- or post- synaptic points
#     anno_layer = ngstbld.AnnotationLayerConfig(name='annos',
#                                        mapping_rules=points )

#     sb = ngstbld.StateBuilder([img_layer, seg_layer, anno_layer], resolution=[4.3,4.3,45])
#     return sb.render_state(render_synapses, return_as='html')

In [ ]:
# x = [3000 , 38000]
# y = [90483, 123190] # extended the boundaries a bit 5/10/22
# z = [980, 3858]
# t1 = lambda b: (b[0]>=x[0]) & (b[0]<=x[1]) & (b[1]>=y[0]) & (b[1]<=y[1]) & (b[2]>=z[0]) & (b[2]<=z[1])

# sid = 648518346494207602
# iln_syn_df = client.materialize.query_table('synapses_jan2022',
#                                                     filter_equal_dict = {'post_pt_root_id': sid},
#                                                     select_columns=['id','post_pt_position'],
#                                                        )
# in_T1_box = iln_syn_df.loc[:,['post_pt_position']].applymap(t1)
# pts_out_of_T1 = iln_syn_df.loc[~in_T1_box.post_pt_position,:]
# pts_in_T1 = iln_syn_df.loc[in_T1_box.post_pt_position,:]
# s = soma_table.loc[sid,:].pt_position
# nearsoma = lambda b: (b[0]>=s[0]+sx[0]) & (b[0]<=s[0]+sx[1]) & (b[1]>=s[1]+sy[0]) & (b[1]<=s[1]+sy[1]) & (b[2]>=s[2]+sz[0]) & (b[2]<=s[2]+sz[1])
# in_soma_box = pts_out_of_T1.loc[:,['post_pt_position']].applymap(nearsoma)
# pts_out_of_T1 = pts_out_of_T1.loc[~in_soma_box.post_pt_position,:]
# if pts_out_of_T1.empty:
#     print('{} is local'.format(sid))
# else: 
#     print('{} has {} inputs outside of T1'.format(sid,pts_out_of_T1.shape[0]))